In [11]:
import sys
!{sys.executable} -m pip install boto3 --upgrade
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install -U -e ..

Requirement already up-to-date: boto3 in /usr/local/lib/python3.7/site-packages (1.14.20)
Obtaining file:///Users/changhonghsu/src/flytekit
  Attempting uninstall: flytekit
    Found existing installation: flytekit 0.10.0b0
    Uninstalling flytekit-0.10.0b0:
      Successfully uninstalled flytekit-0.10.0b0
  Running setup.py develop for flytekit


In [54]:
from flytekit.configuration import set_flyte_config_file, platform
from flytekit.common.tasks.task import SdkTask
from flytekit.sdk.workflow import workflow_class, Input, Output
from flytekit.models.sagemaker.training_job import StoppingCondition

from flytekit.models.sagemaker.hpo_job import HPOJobConfig, HyperparameterTuningObjective
from flytekit.models.sagemaker.parameter_ranges import ParameterRanges, CategoricalParameterRange, ContinuousParameterRange, IntegerParameterRange
from flytekit.common.tasks.presto_task import SdkPrestoTask
from flytekit.sdk.tasks import inputs
from flytekit.sdk.tasks import inputs
from flytekit.sdk.types import Types
from flytekit.sdk.workflow import workflow_class, Input, Output
from flytekit.common.tasks.sagemaker import training_job_task, hpo_job_task
from flytekit.models.sagemaker import training_job as training_job_models, hpo_job as hpo_job_models
from flytekit.sdk.sagemaker import types as _sdk_sagemaker_types
from os import environ
from flytekit.sdk import test_utils as _test_utils
import pandas as pd 

environ["version"] = "training_32"
environ["spec_version"] = "training_32-1"

set_flyte_config_file("/Users/changhonghsu/.flyte/notebook-staging.config")
#set_flyte_config_file("notebook.config")

print("Connected to {}".format(platform.URL.get()))

def print_console_url(exc):
    print("http://{}/console/projects/{}/domains/{}/executions/{}".format(platform.URL.get(), exc.id.project, exc.id.domain, exc.id.name))
    
def print_schema(schema):
    with _test_utils.LocalTestFileSystem() as sandbox:
        # load schema data
        schema.download()
        df = pd.read_parquet(schema.local_path)
    print(df)    
    
    
# Defining the values of some hyperparameters, which will be used by the TrainingJob 
xgboost_hyperparameters = {
    "num_round": "6",
    "base_score": "0.5",
    "booster": "gbtree",
    "csv_weights": "0",
    "dsplit": "row",
    "grow_policy": "depthwise",
    "lambda_bias": "0.0",
    "max_bin": "256",
    "normalize_type": "tree",
    "objective": "reg:linear",
    "one_drop": "0",
    "prob_buffer_row": "1.0",
    "process_type": "default",
    "refresh_leaf": "1",
    "sample_type": "uniform",
    "scale_pos_weight": "1.0",
    "silent": "0",
    "skip_drop": "0.0",
    "tree_method": "auto",
    "tweedie_variance_power": "1.5",
    "updater": "grow_colmaker,prune",
}

# Data transformation task
transform_parquet_to_csv = SdkTask.fetch(project="flytesnacks", domain="development", name="transform_parquet_to_csv", version="24")

environ["version"] = "training_39"
environ["spec_version"] = "training_39-1"

Connected to flyte-staging.lyft.net


# Single-Task Execution Demo
Let's say we just kicked off a workflow like this, in which we wrote a **SageMaker SimpleTrainingJob task** to train **an XGBoost model** on a dataset.
![XGBoost workflow](wf-failed.png "Failed workflow")


### Oops! The TrainingJob task failed 

![Failed Execution](failed_exec.png "Failed exec")

### Let's take a look at the definition of the TrainingJob Task

In [55]:
alg_spec = training_job_models.AlgorithmSpecification(
    input_mode=_sdk_sagemaker_types.InputMode.FILE,
    algorithm_name=_sdk_sagemaker_types.AlgorithmName.XGBOOST,
    # ----->>>> BUGGY IMAGE VERSION !! <<<<------
    algorithm_version="0.72",
    metric_definitions=[
        training_job_models.MetricDefinition(name="Minimize", regex="validation:error")
    ]
)

############################
# Define a TrainingJob task 
############################
xgboost_train_task = training_job_task.SdkSimpleTrainingJobTask(
    training_job_config=training_job_models.TrainingJobConfig(
        instance_type="ml.m4.xlarge",
        instance_count=1,
        volume_size_in_gb=25,
    ),
    algorithm_specification=alg_spec,
    cache_version='3',
    cacheable=True,
)

# Defining inputs for the TrainingJob task
training_inputs={
    "train": "s3://lyft-modelbuilder/test-datasets/pima/train",
    "validation": "s3://lyft-modelbuilder/test-datasets/pima/validation",
    "static_hyperparameters": xgboost_hyperparameters,
    "stopping_condition": StoppingCondition(
        max_runtime_in_seconds=43200,
    ).to_flyte_idl(),
}


# A failed sagemaker task example
# https://flyte-staging.lyft.net/console/projects/flyteexamples/domains/development/executions/tlomtoafvh

### Let's use the task's `register_and_launch()` method to run the TrainingJob Task standalone
... and then you will get a link to the single task execution of our TrainingJob task!

In [56]:
training_exc = xgboost_train_task.register_and_launch("flyteexamples", "development", 
                                                      inputs=training_inputs)
# print_console_url(training_exc)

A working sagemaker task example <br>
https://flyte-staging.lyft.net/console/projects/flyteexamples/domains/development/executions/soxtmrw4am

## Not just for SageMaker tasks
Single task execution **works for other types of task**, too!

For example, we have a **Presto task** to pull input data from our database here. <br> 
We can use single task execution with it for faster iterations as well

### Definition of our `get_train_data2` task (A Presto Task)

In [57]:

get_train_data2 = SdkPrestoTask(
    task_inputs=inputs(),
    #######################
    # BUGGY statement below
    #######################
    statement="""
    SELECT * 
    FROM hive.flyte.datacouncildemo_train
    """,
    output_schema=Types.Schema(),
    discoverable=True,
    discovery_version="3",
)
task_exec = get_train_data2.register_and_launch(
    project="flytekit", domain="development", inputs={"ds": '2020-07-05'})

print("Created execution.")
print_console_url(task_exec)
print("Waiting for execution to complete...")
# task_exec.wait_for_completion()
# print("Done!")

# print_schema(task_exec.outputs['results'])

# a failed example 
# https://flyte-staging.lyft.net/console/projects/flytekit/domains/development/executions/tktijtxmmj

Created execution.
http://flyte-staging.lyft.net/console/projects/flytekit/domains/development/executions/twcd62vcb2
Waiting for execution to complete...


a working example 
http://flyte-staging.lyft.net/console/projects/flytekit/domains/development/executions/k1f136px87

In [51]:
get_validation_data = SdkPrestoTask(
    task_inputs=inputs(),
    statement="""
    SELECT * 
    FROM hive.flyte.datacouncildemo_validation
    """,
    output_schema=Types.Schema(),
    discoverable=True,
    discovery_version="2",
)
task_exec = get_validation_data.register_and_launch(project="flytekit", domain="development", inputs={"ds": '2020-07-05'})
print("Created execution.")
# print_console_url(task_exec)

# No need to run this. it's just a copy of the training data task.
# In a real scenario, we will probably query a huge data set then apply some common algorithm to split the datasets 
#  (e.g. 20-80)
# get_validation_data.register(project="flytesnacks", domain="development", name="get_validation_data", version=environ["version"])

Created execution.
http://flyte-staging.lyft.net/console/projects/flytekit/domains/development/executions/lvs3rxn8zo


# Once you are ready
Register your task and launch them from within your workflow!

In [58]:
get_train_data2.register(project="flytesnacks", domain="development", name="get_train_data", version=environ["version"])
get_validation_data.register(project="flytesnacks", domain="development", name="get_validation_data", version=environ["version"])

@workflow_class()
class TrainingWorkflow(object):    
    
    ################################
    # Invoking the presto tasks you
    # finished iterating on
    ################################
    train_data = get_train_data2()
    validation_data = get_validation_data()
    
    # Transform data
    train_csv = transform_parquet_to_csv(input_parquet=train_data.outputs.results)
    validation_csv = transform_parquet_to_csv(input_parquet=validation_data.outputs.results)
    
    ###############################################
    # Invoking the SageMaker TrainingJob task 
    # you finished iterating on
    ###############################################
    train = xgboost_train_task(
        # Using the input we got from the Presto tasks
        train=train_csv.outputs.output_csv,
        validation=validation_csv.outputs.output_csv,
        static_hyperparameters=xgboost_hyperparameters,
        stopping_condition=StoppingCondition(max_runtime_in_seconds=43200).to_flyte_idl(),
    )
    
    model = Output(train.outputs.model, sdk_type=Types.Blob)
    

In [38]:

TrainingWorkflow.register(project="flyteexamples", domain="development", 
                          name="TrainingWorkflow", version=environ["spec_version"])
TrainingWorkflow_lp = TrainingWorkflow.create_launch_plan()
TrainingWorkflow_lp.register(project="flyteexamples", domain="development", name="TrainingWorkflow", version=environ["spec_version"])


'lp:flyteexamples:development:TrainingWorkflow:training_34-1'

In [59]:
exec = TrainingWorkflow_lp.launch(project="flyteexamples", domain="development", inputs={})
print_console_url(exec)
exec.wait_for_completion()

http://flyte-staging.lyft.net/console/projects/flyteexamples/domains/development/executions/f653dfdf7f43c4de7910


A working example <br> 
https://flyte-staging.lyft.net/console/projects/flyteexamples/domains/development/executions/fa573357670a04d05ba2

# What we've shown ...

... is how single-task execution allows users to 
    - debug and iterate on tasks quickly from within their notebook
    - run single queries on Flyte easily without building an image
    - easily graduate tasks into full-fledged workflows
